In [1]:
# load the dataset
from datasets import load_dataset

kr3 = load_dataset("Wittgensteinian/KR3", name='kr3', split='train')
kr3 = kr3.remove_columns(['__index_level_0__'])

Using custom data configuration Wittgensteinian--KR3-9f7e41b8ab859e44
Reusing dataset parquet (/home/gsdsaml/.cache/huggingface/datasets/parquet/Wittgensteinian--KR3-9f7e41b8ab859e44/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


In [2]:
# drop label==2
kr3_binary = kr3.filter(lambda example: example['Rating'] != 2)

Loading cached processed dataset at /home/gsdsaml/.cache/huggingface/datasets/parquet/Wittgensteinian--KR3-9f7e41b8ab859e44/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901/cache-964db1339d22becb.arrow


In [3]:
# load base model and tokenizer
from transformers import BertForSequenceClassification, BertTokenizer
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [4]:
# map tokenizer
def tokenize_func(example):
    # w/o padding argument --> saved as kr3_tokenized
    # w/ padding=True --> saved as kr3_tokenized_maxpad
    return tokenizer(example['Review'], truncation=True)

kr3_tokenized = kr3_binary.map(tokenize_func, batched=True)

  0%|          | 0/460 [00:00<?, ?ba/s]

In [5]:
# remove and rename columns
kr3_tokenized = kr3_tokenized.remove_columns(['Review']) # tokenized 'Review' is saved in other columns
kr3_tokenized = kr3_tokenized.rename_column('Rating', 'labels')  # BERT in hgf need a parameter named 'labels'
kr3_tokenized.set_format(type='torch')

In [6]:
# save tokenized kr3
kr3_tokenized.save_to_disk('kr3_tokenized')

In [17]:
# how to load
from datasets import Dataset
kr3_loaded = Dataset.load_from_disk('kr3_tokenized_maxpad')